In [1]:
# import sys
# sys.path.insert(1,'/media/mhealthra2/Data/rakib/gits/pytorch-summary/torchsummary')

In [2]:
# from torchsummary2 import summary,summary_string

In [1]:
import torch
from torch.nn.modules import Module
import torch.nn as nn
import torch.nn.functional as F
from torchsummary import summary,summary
from math import floor,ceil
import numpy as np, matplotlib.pyplot as plt, pandas as pd, os

In [8]:
class Net(nn.Module):	
    def __init__(self):
        super(Net,self).__init__()
        self.conv1 = Conv_Gammatone(1,64,81,1000)
#         self.conv1 = nn.Conv1d(1,32,81,stride=1,padding=0,bias=False)
        self.pool = nn.MaxPool1d(2)
#         self.linear1 = nn.Linear(1248,2)
        self.soft = nn.Softmax(dim=0)
    def forward(self,x):
        x = self.conv1(x)
        x = self.pool(x)
#         x = self.linear1(x)
        x = self.soft(x)

In [9]:
net = Net()

In [10]:
wow = summary(net.cuda(),(1,2500))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
    Conv_Gammatone-1             [-1, 64, 2420]             192
         MaxPool1d-2             [-1, 64, 1210]               0
           Softmax-3             [-1, 64, 1210]               0
Total params: 192
Trainable params: 192
Non-trainable params: 0
----------------------------------------------------------------
Input size (MB): 0.01
Forward/backward pass size (MB): 2.36
Params size (MB): 0.00
Estimated Total Size (MB): 2.37
----------------------------------------------------------------


In [10]:
print(wow)

None


In [20]:
for x in net.conv1.named_parameters():
    print(x)

('fc', Parameter containing:
tensor([[  0.0000],
        [  6.0146],
        [ 12.0808],
        [ 18.1991],
        [ 24.3701],
        [ 30.5940],
        [ 36.8713],
        [ 43.2027],
        [ 49.5884],
        [ 56.0291],
        [ 62.5250],
        [ 69.0768],
        [ 75.6849],
        [ 82.3497],
        [ 89.0718],
        [ 95.8517],
        [102.6898],
        [109.5867],
        [116.5428],
        [123.5587],
        [130.6349],
        [137.7719],
        [144.9702],
        [152.2304],
        [159.5529],
        [166.9383],
        [174.3873],
        [181.9001],
        [189.4776],
        [197.1202],
        [204.8284],
        [212.6029],
        [220.4441],
        [228.3528],
        [236.3294],
        [244.3745],
        [252.4888],
        [260.6728],
        [268.9271],
        [277.2523],
        [285.6490],
        [294.1180],
        [302.6596],
        [311.2747],
        [319.9638],
        [328.7275],
        [337.5665],
        [346.4815],
        [35

In [21]:
for x,y in net.named_parameters():
#     print(x)
    print(y.requires_grad)
#     print(x[0], x[1].shape)
#     print(x[1].requires_grad)

True
True
True


In [12]:
import torch
import torch.nn as nn
from torch.autograd import Variable

from collections import OrderedDict
import numpy as np

def register_hook(module):

    def hook(module, input, output):
        class_name = str(module.__class__).split(".")[-1].split("'")[0]
        module_idx = len(summary)
        
        
        m_key = "%s-%i" % (class_name, module_idx + 1)
        summary[m_key] = OrderedDict()
        summary[m_key]["input_shape"] = list(input[0].size())
        summary[m_key]["input_shape"][0] = batch_size
        if isinstance(output, (list, tuple)):
            summary[m_key]["output_shape"] = [
                [-1] + list(o.size())[1:] for o in output
            ]
        else:
            summary[m_key]["output_shape"] = list(output.size())
            summary[m_key]["output_shape"][0] = batch_size

        params = 0
        
        ### updated part
        for (name,weight) in module.named_parameters():
            params += torch.prod(torch.LongTensor(list(weight.shape)))
            summary[m_key]["trainable"] = weight.requires_grad
        ###
           
        ### previously was this
#         if hasattr(module, "weight") and hasattr(module.weight, "size"):
#             params += torch.prod(torch.LongTensor(list(module.weight.size())))
#             summary[m_key]["trainable"] = module.weight.requires_grad
        ###
    
    
        if hasattr(module, "bias") and hasattr(module.bias, "size"):
            params += torch.prod(torch.LongTensor(list(module.bias.size())))
        summary[m_key]["nb_params"] = params

    if (
        not isinstance(module, nn.Sequential)
        and not isinstance(module, nn.ModuleList)
        and not (module == model)
    ):
        hooks.append(module.register_forward_hook(hook))

In [13]:
device="cuda"
if device == "cuda" and torch.cuda.is_available():
    dtype = torch.cuda.FloatTensor
else:
    dtype = torch.FloatTensor

In [14]:
input_size = (1,2500)
if isinstance(input_size, tuple):
    input_size = [input_size]
x = [torch.rand(2, *in_size).type(dtype) for in_size in input_size]

In [15]:
summary = OrderedDict()
hooks = []

In [16]:
summary,hooks

(OrderedDict(), [])

In [17]:
model = net.cuda()
model.apply(register_hook)

Net(
  (conv1): Conv_Gammatone(1, 64, kernel_size=(81,), stride=(1,), bias=False)
  (pool): MaxPool1d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (soft): Softmax(dim=0)
)

In [18]:
batch_size = 2
model(*x)

In [19]:
for h in hooks:
    h.remove()

In [20]:
print("----------------------------------------------------------------")
line_new = "{:>20}  {:>25} {:>15}".format("Layer (type)", "Output Shape", "Param #")
print(line_new)
print("================================================================")
total_params = 0
total_output = 0
trainable_params = 0

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #


In [21]:
for layer in summary:
    print(layer)
    print()

Conv_Gammatone-1

MaxPool1d-2

Softmax-3



In [22]:
for layer in summary:
    print(layer)
    print(summary[layer]["nb_params"])
    # input_shape, output_shape, trainable, nb_params
    line_new = "{:>20}  {:>25} {:>15}".format(
        layer,
        str(summary[layer]["output_shape"]),
        "{0:,}".format(summary[layer]["nb_params"]),
    )
    total_params += summary[layer]["nb_params"]
    total_output += np.prod(summary[layer]["output_shape"])
    if "trainable" in summary[layer]:
        if summary[layer]["trainable"] == True:
            trainable_params += summary[layer]["nb_params"]
    print(line_new)

Conv_Gammatone-1
tensor(192)
    Conv_Gammatone-1              [2, 64, 2420]             192
MaxPool1d-2
0
         MaxPool1d-2              [2, 64, 1210]               0
Softmax-3
0
           Softmax-3              [2, 64, 1210]               0


In [24]:
total_params.numpy()

array(192)

In [243]:
Parameter(torch.Tensor(in_channels, out_channels // groups, *kernel_size))
Parameter(torch.Tensor(1, filters, kernel_size))

NameError: name 'in_channels' is not defined

In [29]:
def hz2mel(hz):
        return 2595 * np.log10(1+hz/700.)
def mel2hz(mel):
    return 700*(10**(mel/2595.0)-1)

In [200]:
Parameter(torch.ones((5,1))*20)

Parameter containing:
tensor([[20.],
        [20.],
        [20.],
        [20.],
        [20.]], requires_grad=True)

In [7]:
from torch.nn.parameter import Parameter
from torch.nn.modules.utils import _single
class Conv_Gammatone(Module):

    __constants__ = ['stride', 'padding', 'dilation', 'groups', 'bias',
                     'padding_mode', 'output_padding', 'in_channels',
                     'out_channels', 'kernel_size']

    def __init__(self, in_channels, out_channels, kernel_size,fsHz, stride=1,
                 padding=0, dilation=1, transposed=False, output_padding=(0,),
                 groups=1, bias=False, padding_mode='zeros',fc=(0,500),beta_val=100,amp_val=10**5,n_order=4):
        super(Conv_Gammatone, self).__init__()
        if in_channels % groups != 0:
            raise ValueError('in_channels must be divisible by groups')
        if out_channels % groups != 0:
            raise ValueError('out_channels must be divisible by groups')
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.filters = out_channels
        self.kernel_size = _single(kernel_size)
        self.stride = _single(stride)
        self.padding = _single(padding)
        self.dilation = _single(dilation)
        self.transposed = transposed
        self.output_padding = output_padding
        self.groups = groups
        self.padding_mode = padding_mode
        self.fsHz = fsHz
        if isinstance(fc,tuple):
            (minf,maxf)=fc
        else:
            minf = 0
            maxf = fc
        self.fc = Parameter(torch.from_numpy(self.mel2hz(np.linspace(self.hz2mel(minf),self.hz2mel(maxf),
                            self.filters,dtype=np.float32))).unsqueeze(-1))
        
        self.beta = Parameter(torch.ones((self.filters,1))*beta_val)
        
        self.amp = Parameter(torch.ones((self.filters,1))*amp_val)
        
        self.n_order = (torch.tensor(n_order,dtype=torch.float))
        
#         self.weight = torch.([self.fc, self.beta, self.amp])
        
        self.register_parameter('bias', None)
    def impulse_gammatone(self):
        device = 0
#         print(self.amp.get_device())
#         print(self.beta.get_device())
#         print(self.fc.get_device())
#         print(self.n_order.get_device())
        
        self.t = torch.arange(0,self.kernel_size[0]/self.fsHz,
                            1/self.fsHz,dtype = torch.float).unsqueeze(-1).transpose(1,0)
    
        self.t = self.t.type(torch.FloatTensor)
        self.n_order = self.n_order.type(torch.FloatTensor)
#         print("device",self.t.get_device())
#         print(self.n_order.get_device())
#         print(self.amp.get_device())
        power = torch.pow(self.t,self.n_order-1)
#         print("power ", power.get_device())
        power = power.to(device = device)
#         print("power ", power.get_device())
        
        at = self.amp.to(device=device)*power
        
#         print("exp")
#         print((-2*torch.tensor(np.pi).to(device)).get_device())
#         print(":/ "torch.mm(self.beta,self.t.to(device)).get_device())
        
        exp = torch.exp(-2*torch.tensor(np.pi).to(device)*torch.mm(self.beta.to(device),self.t.to(device)))
        cos = torch.cos(2*torch.tensor(np.pi).to(device)*torch.mm(self.fc.to(device),self.t.to(device)))
        return at*exp*cos
    def forward(self, input):
        gammatone = self.impulse_gammatone().unsqueeze(1)
        if self.padding_mode == 'circular':
            expanded_padding = ((self.padding[0] + 1) // 2, self.padding[0] // 2)
            return F.conv1d(F.pad(input, expanded_padding, mode='circular'),
                            gammatone, self.bias, self.stride,
                            _single(0), self.dilation, self.groups)
        return F.conv1d(input, gammatone, self.bias, self.stride,
                        self.padding, self.dilation, self.groups)
    
    
#     def reset_parameters(self):
#         init.kaiming_uniform_(self.weight, a=math.sqrt(5))
#         if self.bias is not None:
#             fan_in, _ = init._calculate_fan_in_and_fan_out(self.weight)
#             bound = 1 / math.sqrt(fan_in)
#             init.uniform_(self.bias, -bound, bound)
    def hz2mel(self,hz):
        return 2595 * np.log10(1+hz/700.)
    def mel2hz(self,mel):
        return 700*(10**(mel/2595.0)-1)
    def extra_repr(self):
        s = ('{in_channels}, {out_channels}, kernel_size={kernel_size}'
             ', stride={stride}')
        if self.padding != (0,) * len(self.padding):
            s += ', padding={padding}'
        if self.dilation != (1,) * len(self.dilation):
            s += ', dilation={dilation}'
        if self.output_padding != (0,) * len(self.output_padding):
            s += ', output_padding={output_padding}'
        if self.groups != 1:
            s += ', groups={groups}'
        if self.bias is None:
            s += ', bias=False'
        return s.format(**self.__dict__)

    def __setstate__(self, state):
        super(Conv_Gammatone, self).__setstate__(state)
        if not hasattr(self, 'padding_mode'):
            self.padding_mode = 'zeros'

In [47]:
wow = Conv_Gammatone(1,26,81,1000)

In [45]:
wow.impulse_gammatone().unsqueeze(1).shape

torch.Size([26, 1, 81])

In [365]:
a.get_device()

-1

In [367]:
torch.cuda.is_available()

True